# Lesson 9 - Adding MCP to the Hospital Server

In this final lesson, you will add another agent to the Hospital Server that uses MCP to get access to tools:
- you will first define an MCP server using `FastMCP` that runs using the `stdio` transport (i.e., it runs locally) and exposes one tool;
- you will then update the hospital server to include a second agent;
- you will define the agent using the ToolCallingAgent of Smolagents and pass to it the tool exposed by the MCP server;
- you will finally activate the agent and interact with it using an ACP client.

## 9.1. Output MCP Server

Run this cell to create the file `mcpserver.py` inside the folder `my_acp_project`. The dependencies in the `my_acp_project` have been also updated so you can run this MCP server using `uv run`. Please check the note at the end of the lesson to learn more about these additional dependencies.

## 9.2. Updating the Hospital Server to use MCP

You will add the definition of the second agent `doctor_agent` which helps users find doctors near them. Note that for the MCP server, the command is defined as `uv run`.

In [1]:
!pip install mcpadapt --quiet


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
INFERENCE_SERVER_URL = "http://localhost:8989"
MODEL_NAME = "ibm-granite/granite-3.3-2b-instruct"
API_KEY= "alanliuxiang"

In [ ]:
# %%writefile ../my_acp_project/smolagents_server.py
from collections.abc import AsyncGenerator
from acp_sdk.models import Message, MessagePart
from acp_sdk.server import RunYield, RunYieldResume, Server
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, VisitWebpageTool, ToolCallingAgent, ToolCollection
from mcp import StdioServerParameters

import nest_asyncio
nest_asyncio.apply()

server = Server()

# model = LiteLLMModel(
#     model_id="openai/gpt-4",  
#     max_tokens=2048
# )
from smolagents import OpenAIServerModel

# Configure the model to use LM Studio's local API endpoint
model = OpenAIServerModel(
    model_id= "ibm-granite/granite-3.3-2b-instruct",  # This can be any name, LM Studio will use whatever model you have loaded
    api_base= f"{INFERENCE_SERVER_URL}/v1",  # Default LM Studio API endpoint
    api_key=API_KEY # LM Studio doesn't require an API key by default
)



server_parameters = StdioServerParameters(
    command="uv",
    args=["run", "mcpserver.py"],
    env=None,
)

@server.agent()
async def health_agent(input: list[Message]) -> AsyncGenerator[RunYield, RunYieldResume]:
    "This is a CodeAgent which supports the hospital to handle health based questions for patients. Current or prospective patients can use it to find answers about their health and hospital treatments."
    agent = CodeAgent(tools=[DuckDuckGoSearchTool(), VisitWebpageTool()], model=model)

    prompt = input[0].parts[0].content
    response = agent.run(prompt)

    yield Message(parts=[MessagePart(content=str(response))])

@server.agent()
async def doctor_agent(input: list[Message]) -> AsyncGenerator[RunYield, RunYieldResume]:
    "This is a Doctor Agent which helps users find doctors near them."
    with ToolCollection.from_mcp(server_parameters, trust_remote_code=True) as tool_collection:
        agent = ToolCallingAgent(tools=[*tool_collection.tools], model=model)
        prompt = input[0].parts[0].content
        response = agent.run(prompt)

    yield Message(parts=[MessagePart(content=str(response))])

if __name__ == "__main__":
    server.run(port=8003)

INFO:     Started server process [2769]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8003 (Press CTRL+C to quit)
INFO:     Run started


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I'm based in Atlanta,GA. Are there any Cardiologists near me?                                                   │
│                                                                                                                 │
╰─ OpenAIServerModel - ibm-granite/granite-3.3-2b-instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

INFO:     HTTP Request: POST http://localhost:8989/v1/chat/completions "HTTP/1.1 200 OK"


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'list_doctors' with arguments: {'state': 'GA'}                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'Dr. Sarah Mitchell is a Cardiologist located in        │
│ Atlanta, GA. Her contact details are: (404) 555-2847, s.mitchell@atlantaheart.com, and Emory University         │
│ Hospital. She accepts new patients and is affiliated with Piedmont Atlanta Hospital.'}                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: Dr. Sarah Mitchell is a Cardiologist located in Atlanta, GA. Her contact details are: (404) 555-2847,
s.mitchell@atlantaheart.com, and Emory University Hospital. She accepts new patients and is affiliated with 
Piedmont Atlanta Hospital.

[Step 2: Duration 1.56 seconds| Input tokens: 3,084 | Output tokens: 123]

INFO:     Run completed


INFO:     127.0.0.1:57080 - "POST /runs HTTP/1.1" 200 OK


## 9.3. Run the Hospital ACP x MCP Server

- Open the terminal by running the cell below.
- If the agent is still running from the previous lessons, then you can stop it by typing `CTRL+C`.
- Run the server again by typing:
  - `uv run smolagents_server.py`

## 9.4. Call the ACPxMCP Server

**Note: How to update the `my_acp_project` locally on your machine so you can enable MCP?**
- cd `my_acp_project`
- `uv add mcp smolagents[mcp]`

## 9.4. Resources

- [Smolagents from mcp tool](https://huggingface.co/docs/smolagents/en/reference/tools#smolagents.ToolCollection.from_mcp)
- [MCP short course](https://www.deeplearning.ai/short-courses/mcp-build-rich-context-ai-apps-with-anthropic/)
- [MCP and A2A](https://agentcommunicationprotocol.dev/about/mcp-and-a2a)
- [The doctors json file used accessed through the MCP server](https://raw.githubusercontent.com/nicknochnack/ACPWalkthrough/refs/heads/main/doctors.json)

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To access the <code>my_acp_project</code> folder:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. 

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>
</div>